In [33]:
import numpy as np
import cv2 as cv
import os
import matplotlib.pyplot as plt
CHAIN_ID = "0a998b28bd"
PATH_TO_FOLDER = "/Users/yeojunjie/Documents/NUS Modules/CS3263/Project/spacecraft-pose-estimation/data/images/" + CHAIN_ID + "/"

# The camera intrinsic matrix.
# Source: https://www.drivendata.org/competitions/261/spacecraft-pose-estimation/page/834/#camera-intrinsic-parameters
K = np.array([[5.2125371e+03, 0.0000000e+00, 6.4000000e+02],
              [0.0000000e+00, 6.2550444e+03, 5.1200000e+02],
              [0.0000000e+00, 0.0000000e+00, 1.0000000e+00]])

In [34]:
# Convert a rotation matrix to a quaternion.
# Source: https://en.wikipedia.org/wiki/Rotation_formalisms_in_three_dimensions#Rotation_matrix_↔_quaternion
def rotation_matrix_to_quaternion(R):
    q = np.empty((4,))
    q[0] = np.sqrt(1 + R[0, 0] + R[1, 1] + R[2, 2]) / 2
    q[1] = (R[2, 1] - R[1, 2]) / (4 * q[0])
    q[2] = (R[0, 2] - R[2, 0]) / (4 * q[0])
    q[3] = (R[1, 0] - R[0, 1]) / (4 * q[0])
    return q

In [35]:
LOWES_RATIO = 0.5

# Get the translation vector and rotation quartenion between the i-th and the j-th image in the list.
# (In our usage, i + 1 = j.)
def get_translation_and_rotation(list, i, j):
    assert i < len(list) and j < len(list)

    # Convert the images to grayscale. (Why am I even doing this?)
    gray_i = cv.cvtColor(list[i], cv.COLOR_BGR2GRAY)
    gray_j = cv.cvtColor(list[j], cv.COLOR_BGR2GRAY)

    # Find the keypoints and descriptors with SIFT.
    sift = cv.SIFT_create()
    keypoints_i, descriptors_i = sift.detectAndCompute(gray_i, None)
    keypoints_j, descriptors_j = sift.detectAndCompute(gray_j, None)

    # Create a BFMatcher object and match the descriptors.
    bf = cv.BFMatcher()
    matches = bf.knnMatch(descriptors_i, descriptors_j, k=2)
    # k = 2 means that we get the two best matches.
    # We compare how similar the best match is to the second best match to determine if the best match is unambiguous ('good').

    # Sort the matches by decreasing ambiguity.
    matches.sort(key = lambda x: x[0].distance / x[1].distance, reverse = False)

    # Retain the 15 best matches.
    matches = matches[:15]

    print("At iteration i = " + str(i) + ", the number of good matches is " + str(len(matches)) + ".")
    
    # Extract corresponding points.
    points_i = np.float32([keypoints_i[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    points_j = np.float32([keypoints_j[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Find the fundamental matrix F.
    F, mask = cv.findFundamentalMat(points_i, points_j, cv.FM_RANSAC, 0.1, 0.99)

    # Really dodgy error handling: If F is None, it means that the fundamental matrix could not be calculated.
    if F is None:
        print("Could not calculate the fundamental matrix at iteration i = " + str(i) + ".")
        # Hence, we just return the zero vector and the identity matrix.
        return np.zeros((3, 1)), np.eye(3)
        
    # Calculate the essential matrix E.
    E = np.dot(np.dot(K.T, F), K)

    # Decompose E to get the rotation matrix R and the translation matrix T.
    _, R, T, mask = cv.recoverPose(E, points_i, points_j, K)

    return T, R

In [36]:
# Compose rotation matrices and translation vectors.
def compose_rotation_and_translation(T1, R1, T2, R2):    
    
    # Compose the rotation matrices.
    R = R2 @ R1

    # Compose the translation vectors.
    T = R2 @ T1.T + T2.T

    return T, R

In [37]:
# Load the 100 images.
images = []
for i in range(0, 100):
    PATH_TO_IMAGE = os.path.join(PATH_TO_FOLDER, "{:03d}.png".format(i))
    image = cv.imread(PATH_TO_IMAGE)
    images.append(image)

In [39]:
# Calculate and report the results.

# Create a .csv file with header "chain_id,i,x,y,z,qw,qx,qy,qz".
with open("results.csv", "w") as file:
    file.write("chain_id,i,x,y,z,qw,qx,qy,qz\n")

    # Initialize the translation vector and the rotation matrix.
    # These two variables will be updated as we iterate through the chain of 100 images.
    T = np.zeros((3,))
    R = np.eye(3)

    # Write the line for i = 0.
    file.write(CHAIN_ID + ",0,0.0,0.0,0.0,1.0,0.0,0.0,0.0\n")

    # Iterate through the images.
    for i in range(1, 100):
        # Get the translation vector and the rotation matrix.
        T_i, R_i = get_translation_and_rotation(images, i - 1, i)

        # Compose the translation vector and the rotation matrix.
        T, R = compose_rotation_and_translation(T, R, T_i, R_i)

        # Convert the rotation matrix to a quaternion.
        R_q = rotation_matrix_to_quaternion(R)

        # Write the results to the .csv file.
        file.write("{},{},{},{},{},{},{},{},{}\n".format(CHAIN_ID, i, T[0][0], T[0][1], T[0][2], R_q[0], R_q[1], R_q[2], R_q[3]))


AttributeError: 'tuple' object has no attribute 'sort'